In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch
from utils import *
import numpy as np
#torch.manual_seed(0)

In [2]:
transform = transforms.Compose([
    #transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,
                                          shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,
                                         shuffle=False, num_workers=8)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:

import torch.nn as nn

model = Resnet18_3exits_Pyramidpool().cuda()


In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.6, weight_decay=5e-4)
my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.5)
#model, optimizer = amp.initialize(model, optimizer, opt_level="O2")
#lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,
#                                                              T_0=int(len(trainset)/64*2),
     #                                                         T_mult=1)

In [5]:
%%time
epochs= 75
loss = []
coefficient = [0.2,0.3,0.5]
for epoch in range(epochs):
    if epoch%25==0 and epoch!=0:
        my_lr_scheduler.step()
        print(optimizer.param_groups[0]['lr'])
        
        # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        exit1,exit2,exit3 = model(inputs)
        loss1 = criterion(exit1, labels)
        loss2 = criterion(exit2, labels)
        loss3 = criterion(exit3, labels)
        total_loss = coefficient[0]*loss1+coefficient[1]*loss2+coefficient[2]*loss3
       # with amp.scale_loss(loss, optimizer) as scaled_loss:
        #    scaled_loss.backward()
        total_loss.backward()
        optimizer.step()
        #lr_scheduler.step()
        # print statistics
        running_loss += total_loss.item()
        
        
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss /99))
            loss.append(running_loss)
            running_loss = 0.0

    torch.save(model,"Pyramid_3_exits.pth")
print('Finished Training')

[1,   100] loss: 1.82402
[2,   100] loss: 1.40945
[3,   100] loss: 1.29109
[4,   100] loss: 1.23568
[5,   100] loss: 1.17070
[6,   100] loss: 1.13242
[7,   100] loss: 1.12796
[8,   100] loss: 1.08944
[9,   100] loss: 1.08144
[10,   100] loss: 1.05712
[11,   100] loss: 1.04186
[12,   100] loss: 1.02549
[13,   100] loss: 1.02361
[14,   100] loss: 1.01432
[15,   100] loss: 1.00912
[16,   100] loss: 0.99675
[17,   100] loss: 0.98728
[18,   100] loss: 0.98989
[19,   100] loss: 0.98675
[20,   100] loss: 0.96251
[21,   100] loss: 0.96108
[22,   100] loss: 0.96919
[23,   100] loss: 0.96161
[24,   100] loss: 0.94671
[25,   100] loss: 0.96048
0.05
[26,   100] loss: 0.90146
[27,   100] loss: 0.87557
[28,   100] loss: 0.86510
[29,   100] loss: 0.86203
[30,   100] loss: 0.86229
[31,   100] loss: 0.85275
[32,   100] loss: 0.85356
[33,   100] loss: 0.84624
[34,   100] loss: 0.84859
[35,   100] loss: 0.84601
[36,   100] loss: 0.84981
[37,   100] loss: 0.84263
[38,   100] loss: 0.84499
[39,   100] loss